In [40]:
#imports
import spacy
from spacy.lang.en import English
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS

In [41]:
#get data

data_path = '../data/training_test_data/'
data = pd.read_csv(data_path + 'after_stock_data.csv')
colnames = ['headline','source','label']   
data.columns = colnames

data2 = pd.read_csv(data_path + 'labeled_data.csv')

In [49]:
#data cleansing and concatenation

for i, row in data[data['source'].str.find(' via ') > -1].iterrows():
  data.at[i,'source'] = data.at[i,'source'][:data.at[i,'source'].find(' via ')].strip()

sources = []
headlines = []

for i, row in data2.iterrows():
    sources.append(data2.at[i, 'headlines'][len(data2.at[i, 'headlines'])-data2.at[i, 'headlines'][::-1].find('-'):].strip())
    headlines.append(data2.at[i, 'headlines'][:len(data2.at[i, 'headlines'])-data2.at[i, 'headlines'][::-1].find('-')-1].strip())                                
    
sources[:10] 
headlines[:10] 

clean_data2 = pd.DataFrame({'headline':headlines, 'source':sources, 'label':data2['relevancy']}, columns=colnames)

df_union = pd.concat([data, clean_data2])

df_union['headline'] = df_union['headline'].str.replace("[^a-zA-Z]", " ").replace("\t", " ")

df_union.head(10)

,headline,source,label
0,Apple Amazon YouTube Safer faster amp le...,USA TODAY,0
1,G Stocks Will Amazon Dip Its Toes In,InvestorPlace,0
2,Fear of Amazon Creates a Bargain in FedEx Stock,InvestorPlace,0
3,Amazon Buyout Buzz Draws Options Bulls to Grub...,Schaeffer's Investment Research,0
4,US Stock Market Overview Stocks Surge With O...,FX Empire,0
5,Amazon Shoppers Are Obsessed With This Air Fry...,Cooking Light,0
6,a style font size px href https r sea...,Deadline,0
7,Amazon Stock Will Probably Test but It ...,InvestorPlace,0
8,Scouted Save On Top Rated Dehumidifiers D...,The Daily Beast,0
9,a style font size px href https r sea...,Fashionista,0


In [97]:
#preprocessing
nlp = English()

spacy_stopwords = STOP_WORDS
punct = string.punctuation
    
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

def tokenizer_spacy(headline):
        #filtered_tokenized.append(df_union.at[i, 'headline'])
        #filtered_tokenized.append(row)

    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(str(headline))
    
    #lemmatization improved model
    
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop wordsspacy_stopwords
    return [ str(word).lower().strip() for word in mytokens if str(word).lower().strip() not in spacy_stopwords and str(word).lower().strip() not in punct ]

    # return preprocessed list of tokens
    

In [98]:
bow_vector = CountVectorizer(tokenizer = tokenizer_spacy, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = tokenizer_spacy)

In [99]:
from sklearn.model_selection import train_test_split

X = df_union['headline'] # the features we want to analyze
y= df_union['label'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [100]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('cleaner', <__main__.predictors object at 0x0000026B9FBF7320>), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [101]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7577828397873956
Logistic Regression Precision: 0.7493857493857494
Logistic Regression Recall: 0.8413793103448276


In [ ]:
clf = XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7)

text = Pipeline([
                ('selector', TextSelector(key='description')),
                ('vectorizer', vec_tdidf)
                ])